In [1]:
def enough_money(symbol):
    latest_price = rs.stocks.get_latest_price(symbol)
    latest_price = float(latest_price[0])
    buy_power = float(rs.profiles.load_account_profile(info = 'portfolio_cash'))    
    if buy_power > latest_price:
        return True
    else:
        return False

In [2]:
def place_order(symbol):
    latest_price = rs.stocks.get_latest_price(symbol)
    latest_price = float(latest_price[0])
    #can adjust the purchase price here
    purchase_price = latest_price*1

    print("for ", purchase_price, ", latest price is ", latest_price)
    #places an order based on latest_price
    order_dict = rs.orders.order_buy_limit(symbol=symbol, quantity=1, limitPrice=purchase_price, timeInForce = 'gtc')
    result = order_dict['state']
    #this confirms the order is filled but DOES NOT confirm the stock is PURCHASED
    while result != 'filled':
        get_result = requests.get(endpoint, headers=headers).json()
        result = get_result['state']
    
    #I need to confirm the purchase before setting the sell price
    set_sell_price(symbol, purchase_price=purchase_price) 

In [3]:
def set_sell_price(symbol, purchase_price):
    #aiming for .9% return for each trade
    sell_price = purchase_price * 1.009
    rs.order_sell_limit(symbol=symbol, quantity=1, limitPrice=sell_price, timeInForce='gtc')

In [4]:
def buy_stock(sorted_upward_trending_stocks):
    #iterate through stocks
    for index, row in sorted_upward_trending_stocks.iterrows():        
        #ensures I have enough CASH to purchase stock
        if enough_money(row["symbol"]):
            if place_order(row["symbol"]):
                print("purchased", (row["symbol"]))
                #remove symbol from dataframe
                return None      